In [1]:
import torch
import numpy as np
import pandas as pd
import skimage
import matplotlib.pyplot as plt
import imp
import core
import utils
%matplotlib inline

In [2]:
wd = 'E:/new_data/kaggle/planet/'
train_set = pd.read_csv(wd + 'train_v2.csv')
train_set['tags'] = train_set['tags'].apply(lambda x: x.split(' '))
test_set = pd.read_csv(wd+'sample_submission_v2.csv')
train_tags = ['clear', 'partly_cloudy', 'haze', 'cloudy', 'primary', 'agriculture', 'road', 'water',
             'cultivation', 'habitation', 'bare_ground', 'selective_logging', 'artisinal_mine', 
              'blooming', 'slash_burn', 'conventional_mine', 'blow_down']
label_map = {l: i for i, l in enumerate(train_tags)}
inv_label_map = {i: l for l, i in label_map.items()}

In [176]:
# 载入数据
from sklearn.model_selection import train_test_split
from time import time
file_all = train_set['image_name'].values
y_all = utils.get_y(train_set['tags'].values, label_map)
x_tr, x_vl, y_tr, y_vl = train_test_split(file_all, y_all, test_size=0.8, random_state=int(time()))

In [215]:
from torch import nn, optim
from torch.autograd import Variable
from torchvision import transforms
import random
from PIL import Image
from time import time
import mydataset
imp.reload(utils)
imp.reload(core)
imp.reload(mydataset)
import gc
for i in range(10):
    gc.collect()

# 超参数
batch_size = 256
pic_size = (64, 64)
learning_rate = 1e-3
num_epoches = 1000
tolerance = 15
lr_tolerance = 7
best_model = core.BestModel()
transform1 = transforms.Compose([
#     transforms.RandomCrop(pic_size),
    transforms.RandomHorizontalFlip(),
])
transform2 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
degrees = [0, 45, 90, 135, 180, 225, 270, 315]
rotate_range = (0, 45)
def transform_tr(img, pic_size):
    img = img.resize(pic_size)
    img = img.transpose(Image.FLIP_LEFT_RIGHT)  if random.randint(0, 1) > .5 else img
    img = img.transpose(Image.FLIP_TOP_BOTTOM)  if random.randint(0, 1) > .5 else img
    img.rotate(np.random.random() *  (rotate_range[1] - rotate_range[0]) + rotate_range[0])
    img_tensor = transform2(img)
    return img_tensor

def transform_vl(img, pic_size):
    img = img.resize(pic_size)
    img_tensor = transform2(img)
    return img_tensor

probs1 = [0.1, 0.2, 0.4, 0.4, 0.1, 0.1, 0.2, 0.2, 0.4, 0.4, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
probs = [0.5, 0.6, 0.8, 0.8, 0.5, 0.5, 0.6, 0.6, 0.8, 0.8, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [216]:
# 训练
model = core.MyNet(17).cuda()
# model = resnet.ResNet(resnet.BasicBlock, [2, 2, 2, 2], num_classes=17)
criterion = nn.BCELoss(weight=None).cuda()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

estimator = core.Estimator(model, criterion, optimizer)
for epoch in range(num_epoches):
    time_st = time()
    train_loader = utils.weighted_train_loader(x_tr, y_tr, probs, transform_tr, batch_size, pic_size)
    loss_tr = estimator.train(train_loader)
    
    val_loader = utils.valid_loader(x_vl, y_vl, transform_vl, batch_size, pic_size)
    loss_vl, f2_vl = estimator.validate(val_loader)
    best_model.update(loss_vl.avg, f2_vl, estimator.model)
    
    if epoch > lr_tolerance and best_model.lrcount > lr_tolerance:
        lr = optimizer.param_groups[0]['lr']
#         new_lr = 1e-3 if lr < 1e-5 else lr * 0.1
        new_lr = lr * 0.1
        optimizer.param_groups[0]['lr'] = new_lr
        print('New Learn Rate: {}!'.format(new_lr))
        best_model.lrcount = 0
        
    if epoch > tolerance and best_model.nobetter > tolerance:
        best_model.save('./alexnet-709-1.pth')
        print('Early Stop in Epoch: {}, Best Val-Loss: {:.6f}, Best F2: {:.6f}'.format(
            epoch+1, best_model.best_loss, best_model.best_f2.value(bestf2=True)))
        break
        
    print('[{}/{}] {}s, Loss: {:.4f}, Val-Loss: {:.4f}, Best Val-Loss: {:.4f}, Val-F2: {:.2f}'.format(
        epoch+1,
        num_epoches,
        int(time() - time_st),
        loss_tr.avg,
        loss_vl.avg,
        best_model.best_loss,
        f2_vl.value(0.3)
    ))
    gc.collect()

Update Model!
[1/1000] 662s, Loss: 0.6187, Val-Loss: 0.4695, Best Val-Loss: 0.4695, Val-F2: 0.49
Update Model!
[2/1000] 120s, Loss: 0.3541, Val-Loss: 0.2289, Best Val-Loss: 0.2289, Val-F2: 0.78
Update Model!
[3/1000] 103s, Loss: 0.2116, Val-Loss: 0.1695, Best Val-Loss: 0.1695, Val-F2: 0.82
[4/1000] 97s, Loss: 0.1917, Val-Loss: 0.1792, Best Val-Loss: 0.1695, Val-F2: 0.82
Update Model!
[5/1000] 95s, Loss: 0.1865, Val-Loss: 0.1678, Best Val-Loss: 0.1678, Val-F2: 0.83
Update Model!
[6/1000] 94s, Loss: 0.1753, Val-Loss: 0.1475, Best Val-Loss: 0.1475, Val-F2: 0.85
[7/1000] 93s, Loss: 0.1761, Val-Loss: 0.1547, Best Val-Loss: 0.1475, Val-F2: 0.84
[8/1000] 93s, Loss: 0.1705, Val-Loss: 0.1555, Best Val-Loss: 0.1475, Val-F2: 0.84
Update Model!
[9/1000] 93s, Loss: 0.1644, Val-Loss: 0.1403, Best Val-Loss: 0.1403, Val-F2: 0.86
[10/1000] 93s, Loss: 0.1641, Val-Loss: 0.1416, Best Val-Loss: 0.1403, Val-F2: 0.86
[11/1000] 93s, Loss: 0.1596, Val-Loss: 0.1479, Best Val-Loss: 0.1403, Val-F2: 0.86
[12/1000]

J:\Anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


[26/1000] 93s, Loss: 0.1415, Val-Loss: 0.1307, Best Val-Loss: 0.1246, Val-F2: 0.87
[27/1000] 93s, Loss: 0.1403, Val-Loss: 0.1271, Best Val-Loss: 0.1246, Val-F2: 0.87
Update Model!
[28/1000] 93s, Loss: 0.1375, Val-Loss: 0.1189, Best Val-Loss: 0.1189, Val-F2: 0.89
[29/1000] 93s, Loss: 0.1369, Val-Loss: 0.1211, Best Val-Loss: 0.1189, Val-F2: 0.88
Update Model!
[30/1000] 93s, Loss: 0.1314, Val-Loss: 0.1185, Best Val-Loss: 0.1185, Val-F2: 0.89
[31/1000] 93s, Loss: 0.1346, Val-Loss: 0.1197, Best Val-Loss: 0.1185, Val-F2: 0.88
[32/1000] 93s, Loss: 0.1373, Val-Loss: 0.1244, Best Val-Loss: 0.1185, Val-F2: 0.88
[33/1000] 93s, Loss: 0.1354, Val-Loss: 0.1238, Best Val-Loss: 0.1185, Val-F2: 0.88
[34/1000] 93s, Loss: 0.1326, Val-Loss: 0.1203, Best Val-Loss: 0.1185, Val-F2: 0.88
[35/1000] 93s, Loss: 0.1290, Val-Loss: 0.1218, Best Val-Loss: 0.1185, Val-F2: 0.88
[36/1000] 93s, Loss: 0.1325, Val-Loss: 0.1323, Best Val-Loss: 0.1185, Val-F2: 0.88
[37/1000] 93s, Loss: 0.1329, Val-Loss: 0.1207, Best Val-Los

In [6]:
from torch import nn, optim
from torch.autograd import Variable
from torchvision import transforms
import random
from PIL import Image
from time import time
import mydataset
import gc
imp.reload(utils)
imp.reload(core)
imp.reload(mydataset)
for i in range(10):
    gc.collect()

# 超参数
batch_size = 256
pic_size = (64, 64)
learning_rate = 1e-3
num_epoches = 1000
tolerance = 15
lr_tolerance = 7
transform1 = transforms.Compose([
    transforms.RandomHorizontalFlip(),
])
transform2 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

def transform_tr(img, pic_size):
    img = img.resize(pic_size)
    img = img.transpose(Image.FLIP_LEFT_RIGHT)  if random.randint(0, 1) > .5 else img
    img = img.transpose(Image.FLIP_TOP_BOTTOM)  if random.randint(0, 1) > .5 else img
    img.rotate(np.random.random() * 45)
    img_tensor = transform2(img)
    return img_tensor

def transform_vl(img, pic_size):
    img = img.resize(pic_size)
    img_tensor = transform2(img)
    return img_tensor

In [7]:
n_splits = 5
model_name = 'weightedNN1'
date = '710'
probs1 = [0.1, 0.2, 0.4, 0.4, 0.1, 0.1, 0.2, 0.2, 0.4, 0.4, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
probs = [0.5, 0.6, 0.8, 0.8, 0.5, 0.5, 0.6, 0.6, 0.8, 0.8, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

from sklearn.model_selection import KFold
from itertools import islice
file_all = train_set['image_name'].values
test_file_all = test_set['image_name'].values
y_all = utils.get_y(train_set['tags'].values, label_map)
pred_tr = np.zeros((file_all.shape[0], 17))
pred_ts = np.zeros((test_file_all.shape[0], 17))
kf = KFold(n_splits = n_splits)

k_now = 0
for i_tr, i_vl in islice(kf.split(y_all), 0, None):
    model = core.MyNet(17).cuda()
    criterion = nn.BCELoss(weight=None).cuda()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    estimator = core.Estimator(model, criterion, optimizer)
    best_model = core.BestModel()
    
    for epoch in range(num_epoches):
        time_st = time()
        # 训练
        train_loader = utils.weighted_train_loader(file_all[i_tr], y_all[i_tr], probs, transform_tr, batch_size, pic_size)
        loss_tr = estimator.train(train_loader)

        # 验证
        val_loader = utils.valid_loader(file_all[i_vl], y_all[i_vl], transform_vl, batch_size, pic_size)
        loss_vl, f2_vl = estimator.validate(val_loader)
        best_model.update(loss_vl.avg, f2_vl, estimator.model)
        
        # 若验证结果提升缓慢，减小学习率
        if epoch > lr_tolerance and best_model.lrcount > lr_tolerance:
            optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] * 0.1
            print('New Learn Rate: {}!'.format(optimizer.param_groups[0]['lr']))
            best_model.lrcount = 0
        
        # 若验证结果不再提升，保存模型、验证结果、预测结果，跳出迭代
        if epoch > tolerance and best_model.nobetter > tolerance:
            print('Early Stop in Epoch: {}, Best Val-Loss: {:.6f}, Best F2: {:.6f}'.format(
                epoch+1, best_model.best_loss, best_model.best_f2.value(bestf2=True)))
            pred_tr[i_vl, :] = f2_vl.preds
            best_model.save('./model{}-date{}-kf{}.pth'.format(model_name, date, k_now+1))
                                                               
            tst_loader = utils.test_loader(test_file_all, transform_vl, batch_size, pic_size)
            estimator.model.load_state_dict(best_model.best_model)                                       
            pred_ts_temp = estimator.predict(tst_loader)
            pred_ts =  pred_ts_temp / float(n_splits)
            np.save('./model{}_date{}_pred_train_kf{}.npy'.format(model_name, date, k_now+1), pred_tr)
            np.save('./model{}_date{}_pred_test_kf{}.npy'.format(model_name, date, k_now+1), pred_ts_temp)
            break
        
        # 打印每一次迭代的训练验证成绩
        print('{} [{}/{}] {}s, Loss: {:.4f}, Val-Loss: {:.4f}, Best Val-Loss: {:.4f}, Val-F2: {:.2f}'.format(
            k_now+1, epoch+1, num_epoches, int(time() - time_st), loss_tr.avg, loss_vl.avg, 
            best_model.best_loss, f2_vl.value(0.3)))
        gc.collect()
    k_now += 1

# 序列化验证和预测结果，用于stacking
np.save('./model{}_date{}_pred_train.npy'.format(model_name, date), pred_tr)
np.save('./model{}_date{}_pred_test.npy'.format(model_name, date), pred_ts)

Update Model!
1 [1/1000] 514s, Loss: 0.3404, Val-Loss: 0.1833, Best Val-Loss: 0.1833, Val-F2: 0.81
Update Model!
1 [2/1000] 222s, Loss: 0.1709, Val-Loss: 0.1414, Best Val-Loss: 0.1414, Val-F2: 0.86
Update Model!
1 [3/1000] 141s, Loss: 0.1575, Val-Loss: 0.1336, Best Val-Loss: 0.1336, Val-F2: 0.87
Update Model!
1 [4/1000] 116s, Loss: 0.1502, Val-Loss: 0.1334, Best Val-Loss: 0.1334, Val-F2: 0.86
Update Model!
1 [5/1000] 100s, Loss: 0.1453, Val-Loss: 0.1264, Best Val-Loss: 0.1264, Val-F2: 0.88
Update Model!
1 [6/1000] 95s, Loss: 0.1409, Val-Loss: 0.1212, Best Val-Loss: 0.1212, Val-F2: 0.88
1 [7/1000] 94s, Loss: 0.1395, Val-Loss: 0.1232, Best Val-Loss: 0.1212, Val-F2: 0.89
Update Model!
1 [8/1000] 94s, Loss: 0.1334, Val-Loss: 0.1159, Best Val-Loss: 0.1159, Val-F2: 0.90
Update Model!
1 [9/1000] 96s, Loss: 0.1317, Val-Loss: 0.1109, Best Val-Loss: 0.1109, Val-F2: 0.89
1 [10/1000] 96s, Loss: 0.1313, Val-Loss: 0.1145, Best Val-Loss: 0.1109, Val-F2: 0.89
1 [11/1000] 94s, Loss: 0.1279, Val-Loss: 0

J:\Anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


1 [18/1000] 95s, Loss: 0.1197, Val-Loss: 0.1073, Best Val-Loss: 0.1033, Val-F2: 0.90
1 [19/1000] 99s, Loss: 0.1179, Val-Loss: 0.1043, Best Val-Loss: 0.1033, Val-F2: 0.90
Update Model!
1 [20/1000] 97s, Loss: 0.1175, Val-Loss: 0.1007, Best Val-Loss: 0.1007, Val-F2: 0.91
Update Model!
1 [21/1000] 95s, Loss: 0.1151, Val-Loss: 0.1006, Best Val-Loss: 0.1006, Val-F2: 0.91
1 [22/1000] 95s, Loss: 0.1149, Val-Loss: 0.1009, Best Val-Loss: 0.1006, Val-F2: 0.90
1 [23/1000] 116s, Loss: 0.1149, Val-Loss: 0.1020, Best Val-Loss: 0.1006, Val-F2: 0.91
Update Model!
1 [24/1000] 105s, Loss: 0.1131, Val-Loss: 0.1003, Best Val-Loss: 0.1003, Val-F2: 0.91
Update Model!
1 [25/1000] 98s, Loss: 0.1131, Val-Loss: 0.0995, Best Val-Loss: 0.0995, Val-F2: 0.91
Update Model!
1 [26/1000] 96s, Loss: 0.1126, Val-Loss: 0.0970, Best Val-Loss: 0.0970, Val-F2: 0.91
1 [27/1000] 94s, Loss: 0.1116, Val-Loss: 0.1019, Best Val-Loss: 0.0970, Val-F2: 0.90
1 [28/1000] 94s, Loss: 0.1106, Val-Loss: 0.1028, Best Val-Loss: 0.0970, Val-F2

Update Model!
2 [12/1000] 93s, Loss: 0.1256, Val-Loss: 0.1083, Best Val-Loss: 0.1083, Val-F2: 0.90
2 [13/1000] 93s, Loss: 0.1257, Val-Loss: 0.1136, Best Val-Loss: 0.1083, Val-F2: 0.89
Update Model!
2 [14/1000] 93s, Loss: 0.1243, Val-Loss: 0.1044, Best Val-Loss: 0.1044, Val-F2: 0.90
2 [15/1000] 99s, Loss: 0.1211, Val-Loss: 0.1066, Best Val-Loss: 0.1044, Val-F2: 0.90
Update Model!
2 [16/1000] 99s, Loss: 0.1210, Val-Loss: 0.1021, Best Val-Loss: 0.1021, Val-F2: 0.91
2 [17/1000] 99s, Loss: 0.1193, Val-Loss: 0.1078, Best Val-Loss: 0.1021, Val-F2: 0.90
Update Model!
2 [18/1000] 99s, Loss: 0.1199, Val-Loss: 0.1011, Best Val-Loss: 0.1011, Val-F2: 0.91
2 [19/1000] 98s, Loss: 0.1174, Val-Loss: 0.1024, Best Val-Loss: 0.1011, Val-F2: 0.91
2 [20/1000] 99s, Loss: 0.1162, Val-Loss: 0.1033, Best Val-Loss: 0.1011, Val-F2: 0.90
2 [21/1000] 99s, Loss: 0.1177, Val-Loss: 0.1051, Best Val-Loss: 0.1011, Val-F2: 0.90
Update Model!
2 [22/1000] 99s, Loss: 0.1164, Val-Loss: 0.0989, Best Val-Loss: 0.0989, Val-F2: 

Update Model!
3 [18/1000] 99s, Loss: 0.1209, Val-Loss: 0.1016, Best Val-Loss: 0.1016, Val-F2: 0.90
Update Model!
3 [19/1000] 96s, Loss: 0.1183, Val-Loss: 0.1010, Best Val-Loss: 0.1010, Val-F2: 0.91
3 [20/1000] 95s, Loss: 0.1184, Val-Loss: 0.1029, Best Val-Loss: 0.1010, Val-F2: 0.90
3 [21/1000] 95s, Loss: 0.1159, Val-Loss: 0.1032, Best Val-Loss: 0.1010, Val-F2: 0.90
3 [22/1000] 95s, Loss: 0.1148, Val-Loss: 0.1037, Best Val-Loss: 0.1010, Val-F2: 0.91
3 [23/1000] 95s, Loss: 0.1147, Val-Loss: 0.1029, Best Val-Loss: 0.1010, Val-F2: 0.90
Update Model!
3 [24/1000] 95s, Loss: 0.1145, Val-Loss: 0.0986, Best Val-Loss: 0.0986, Val-F2: 0.91
3 [25/1000] 95s, Loss: 0.1135, Val-Loss: 0.1023, Best Val-Loss: 0.0986, Val-F2: 0.90
Update Model!
3 [26/1000] 95s, Loss: 0.1127, Val-Loss: 0.0982, Best Val-Loss: 0.0982, Val-F2: 0.91
Update Model!
3 [27/1000] 95s, Loss: 0.1128, Val-Loss: 0.0977, Best Val-Loss: 0.0977, Val-F2: 0.91
3 [28/1000] 95s, Loss: 0.1104, Val-Loss: 0.1006, Best Val-Loss: 0.0977, Val-F2: 

4 [33/1000] 109s, Loss: 0.1095, Val-Loss: 0.0981, Best Val-Loss: 0.0968, Val-F2: 0.91
Update Model!
4 [34/1000] 101s, Loss: 0.1078, Val-Loss: 0.0952, Best Val-Loss: 0.0952, Val-F2: 0.91
4 [35/1000] 100s, Loss: 0.1075, Val-Loss: 0.0990, Best Val-Loss: 0.0952, Val-F2: 0.91
4 [36/1000] 99s, Loss: 0.1066, Val-Loss: 0.0985, Best Val-Loss: 0.0952, Val-F2: 0.91
4 [37/1000] 105s, Loss: 0.1055, Val-Loss: 0.0954, Best Val-Loss: 0.0952, Val-F2: 0.91
4 [38/1000] 119s, Loss: 0.1063, Val-Loss: 0.0962, Best Val-Loss: 0.0952, Val-F2: 0.91
4 [39/1000] 132s, Loss: 0.1061, Val-Loss: 0.0963, Best Val-Loss: 0.0952, Val-F2: 0.91
4 [40/1000] 113s, Loss: 0.1045, Val-Loss: 0.0972, Best Val-Loss: 0.0952, Val-F2: 0.91
4 [41/1000] 100s, Loss: 0.1040, Val-Loss: 0.0960, Best Val-Loss: 0.0952, Val-F2: 0.91
Update Model!
4 [42/1000] 96s, Loss: 0.1037, Val-Loss: 0.0948, Best Val-Loss: 0.0948, Val-F2: 0.91
Update Model!
4 [43/1000] 94s, Loss: 0.1036, Val-Loss: 0.0940, Best Val-Loss: 0.0940, Val-F2: 0.91
4 [44/1000] 94s

Update Model!
5 [26/1000] 97s, Loss: 0.1134, Val-Loss: 0.0977, Best Val-Loss: 0.0977, Val-F2: 0.91
Update Model!
5 [27/1000] 98s, Loss: 0.1110, Val-Loss: 0.0973, Best Val-Loss: 0.0973, Val-F2: 0.91
5 [28/1000] 98s, Loss: 0.1108, Val-Loss: 0.0980, Best Val-Loss: 0.0973, Val-F2: 0.91
Update Model!
5 [29/1000] 97s, Loss: 0.1094, Val-Loss: 0.0955, Best Val-Loss: 0.0955, Val-F2: 0.91
5 [30/1000] 97s, Loss: 0.1097, Val-Loss: 0.1047, Best Val-Loss: 0.0955, Val-F2: 0.90
5 [31/1000] 98s, Loss: 0.1092, Val-Loss: 0.0999, Best Val-Loss: 0.0955, Val-F2: 0.91
5 [32/1000] 98s, Loss: 0.1100, Val-Loss: 0.0986, Best Val-Loss: 0.0955, Val-F2: 0.91
Update Model!
5 [33/1000] 98s, Loss: 0.1077, Val-Loss: 0.0935, Best Val-Loss: 0.0935, Val-F2: 0.91
5 [34/1000] 97s, Loss: 0.1083, Val-Loss: 0.0991, Best Val-Loss: 0.0935, Val-F2: 0.91
5 [35/1000] 97s, Loss: 0.1062, Val-Loss: 0.0974, Best Val-Loss: 0.0935, Val-F2: 0.91
5 [36/1000] 97s, Loss: 0.1074, Val-Loss: 0.1010, Best Val-Loss: 0.0935, Val-F2: 0.91
5 [37/100

In [8]:
pred_ts

array([[  1.99996424e-01,   1.80484967e-06,   5.19261948e-07, ...,
          1.48035688e-03,   7.84337044e-07,   1.41288929e-06],
       [  1.99879313e-01,   3.24466564e-06,   9.08035669e-05, ...,
          5.88273906e-07,   4.21964216e-08,   2.00709575e-05],
       [  2.52995960e-05,   1.99938583e-01,   1.50089822e-05, ...,
          1.09236244e-05,   3.08093576e-08,   4.12015579e-06],
       ..., 
       [  1.96759224e-01,   9.77026997e-05,   1.60461050e-03, ...,
          1.10387150e-04,   4.36866743e-05,   3.28870188e-04],
       [  1.99969649e-01,   4.74843429e-05,   4.13566922e-06, ...,
          1.47761256e-02,   9.52786009e-05,   5.48756041e-05],
       [  1.99997425e-01,   7.81339622e-07,   7.13423060e-07, ...,
          3.31050366e-02,   6.19326556e-07,   9.38094454e-06]])

In [29]:
from itertools import islice
cc = np.arange(10)
kff = KFold(n_splits = n_splits)
for ii_tr, ii_vl in islice(kff.split(cc), 0, None):
    print(ii_tr, ii_vl)

[2 3 4 5 6 7 8 9] [0 1]
[0 1 4 5 6 7 8 9] [2 3]
[0 1 2 3 6 7 8 9] [4 5]
[0 1 2 3 4 5 8 9] [6 7]
[0 1 2 3 4 5 6 7] [8 9]


In [28]:
np.save('./model{}_date{}_kf0_pred_train.npy'.format(model_name, date), pred_tr)
np.save('./model{}_date{}_kf0_pred_test.npy'.format(model_name, date), pred_ts)

In [48]:
imp.reload(utils)
submit_df = utils.to_submit(pred_ts, th, test_set, inv_label_map)
submit_df.to_csv('./submit/model{}_date{}_no{}.csv'.format(model_name, date, 1), index=False)

In [46]:
th1 = utils.f2_opti_score(y_all, pred_tr, thresholds = np.arange(0, 1, 0.01), num_classes=17)
th2 = utils.f2_opti_score(y_all, pred_tr, thresholds = np.arange(1, 0, -0.01), num_classes=17)
th = (th1 + th2) / 2.0
print(utils.f2_score(y_all, pred_tr, th))

0.92196563004229037

In [41]:
imp.reload(core)
imp.reload(utils)
imp.reload(mydataset)

pred_tr = np.zeros((file_all.shape[0], 17))
pred_ts = np.zeros((test_file_all.shape[0], 17))
model = core.MyNet(17).cuda()
criterion = nn.BCELoss(weight=None).cuda()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
estimator = core.Estimator(model, criterion, optimizer)
k_now = 1
for i_tr, i_vl in islice(kf.split(y_all), 0, None):
    best_model = core.BestModel()
    estimator.model.load_state_dict(torch.load('modelweightedNN1-date710-kf{}.pth'.format(k_now)))
    val_loader = utils.valid_loader(file_all[i_vl], y_all[i_vl], transform_vl, batch_size, pic_size)
    loss_vl, f2_vl = estimator.validate(val_loader)
    pred_tr[i_vl] = f2_vl.preds
    
    tst_loader = utils.test_loader(test_file_all, transform_vl, batch_size, pic_size)
    pred_ts += estimator.predict(tst_loader) / 5.0
    k_now += 1

-----------
-----------
-----------
-----------
-----------


In [44]:
pred_ts.shape

(61191, 17)